In [1]:
import time
import math
import numpy as np
import torch
import random
from torch import nn,optim
import torch.nn.functional as F

import sys

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# 获取数据

In [2]:
import zipfile
with zipfile.ZipFile('./Datasets/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars=f.read().decode('utf-8')
corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars=corpus_chars[0:10000]
idx_to_char=list(set(corpus_chars))
char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
vocab_size=len(char_to_idx)
corpus_indices=[char_to_idx[char] for char in corpus_chars]

# ONEHOT向量

In [3]:
def one_hot(x,n_class,dtype=torch.float32):
    # X shape: (batch), output shape: (batch, n_class)
    x=x.long()
    res=torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res

x=torch.tensor([0,2])
one_hot(x,vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [4]:
def to_onehot(X,n_class):
    # X shape: (batch, seq_len), output: seq_len elements of (batch,n_class)
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]
X=torch.arange(10).view(2,5)
inputs=to_onehot(X,vocab_size)
print(len(inputs),inputs[0].shape)

5 torch.Size([2, 1027])


# 初始化模型参数

In [5]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)
def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    #隐藏层参数
    W_xh=_one((num_inputs,num_hiddens))
    W_hh=_one((num_hiddens,num_hiddens))
    b_h=torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))
    #输出层参数
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])

will use cuda


# 定义模型

In [6]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

In [7]:
def rnn(inputs,state,params):
   # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh,W_hh,b_h,W_hq,b_q=params
    H,=state
    outputs=[]
    for X in inputs:
        H=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [8]:
state=init_rnn_state(X.shape[0],num_hiddens,device)
inputs=to_onehot(X.to(device),vocab_size)
params=get_params()
outputs,state_new=rnn(inputs,state,params)
print(len(outputs),outputs[0].shape,state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


# 定义预测函数

In [9]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state=init_rnn_state(1,num_hiddens,device)
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X=to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state)=rnn(X,state,params)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [10]:
predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'分开斗死节国黄承屋年铜真'

# 裁剪梯度

In [11]:
def grad_clipping(params,theta,device):
    norm=torch.tensor([0.0],device=device)
    for param in params:
        norm+=(param.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm>theta:
        for param in params:
            param.grad.data*=(theta/norm)

# 困惑度

最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正⽆穷；
基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数

# 定义模型训练函数

In [12]:
def data_iter_random(corpus_indices,batch_size,num_steps,device=None):
    # 减1是因为输出的索引x是相应输⼊的索引y加1
    num_examples=(len(corpus_indices)-1)//num_steps
    epoch_size=num_examples//batch_size
    example_indices=list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(epoch_size):
        i=i*batch_size
        batch_indices=example_indices[i:i+batch_size]
        X=[_data(j*num_steps) for j in batch_indices]
        Y=[_data(j*num_steps+1) for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device),torch.tensor(Y,dtype=torch.float32,device=device)

In [13]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y

In [14]:
def sgd(params,lr,batch_size):
    for param in params:
        param.data-=lr*param.grad/batch_size

In [23]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,is_random_iter,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
    if is_random_iter:
        data_iter_fn=data_iter_random
    else:
        data_iter_fn=data_iter_consecutive
    params=get_params()
    loss=nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:# 如使⽤相邻采样，在epoch开始时初始化隐藏状态
            state=init_rnn_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter:
            if is_random_iter:# 如使⽤随机采样，在每个⼩批量更新前初始化隐藏状态
                state=init_rnn_state(batch_size,num_hiddens,device)
            else:
                # 否则需要使⽤detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖⼀次迭代读取的⼩批量序列(防⽌梯度计算开销太⼤）
                for s in state:
                    s.detach_()
            inputs=to_onehot(X,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params,clipping_theta,device)
            sgd(params,lr,1)
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
        if (epoch+1)%pred_period==0:
            print('epoch %d,perplexity %f,time %.2f sec'%(epoch+1,math.exp(l_sum/n),time.time()-start))
            for prefix in prefixes:
                print('-',predict_rnn(prefix,pred_len,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx))

# 训练模型并创造歌词

In [24]:
num_epochs,num_steps,batch_size,lr,clipping_theta=1000,35,32,1e2,1e-2
pred_period,pred_len,prefixes=50,50,['分开','不分开']

In [22]:
train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,True,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes)

epoch 50,perplexity 69.312167,time 0.39 sec
- 分开 我想要这爱 我不能你想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
- 不分开  想要你有 我不要再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
epoch 100,perplexity 10.031016,time 0.38 sec
- 分开 我想想这生活 后知后觉 我该好好生活 我知无好生活 后知不觉 你已经离开我 不知不觉 我已了这节奏
- 不分开堡 我爱你这辈 我已往事些 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 
epoch 150,perplexity 2.802593,time 0.39 sec
- 分开 有子都不  谁在空壶 是小村外的溪边 默默等待 娘子 一壶棍酒 再指 我已 我静情再想 我不能 你
- 不分开简 我不能再想 我不 我不 我不能再想你 爱情不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 200,perplexity 1.573672,time 0.39 sec
- 分开 有直都什一 三色三斗奇 这里什么奇怪的事都有 包括像猫的狗 印地安老斑鸠 平常话不多 除非是乌鸦抢
- 不分开扫把单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不了口 周杰伦
epoch 250,perplexity 1.311683,time 0.39 sec
- 分开 一直用它心仪的母斑鸠 印地安老斑鸠 腿短毛不多 几天都没有喝水也能活 脑袋瓜有一点秀逗 猎物死了它
- 不分开期 我叫你爸 你来我妈 这样对吗夫 这样 他念念依旧词屋 折一枝杨柳 你在那里 在小村外的溪边河口默


In [25]:
train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,False,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes)

epoch 50,perplexity 59.182191,time 0.37 sec
- 分开 我想要这样 我不能我想 我不能我想 我不能我想 我不能我想 我不能我想 我不能我想 我不能我想 我
- 不分开 我不要再可 我不要我想 我不能我想 我不能我想 我不能我想 我不能我想 我不能我想 我不能我想 我
epoch 100,perplexity 6.698817,time 0.39 sec
- 分开 我不要再想 我不要再想 我不能 爱情我的见活 不话去对 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼
- 不分开柳 你怎么这 我 靠带你的让快 像发球 瞎不了 什么就有 在对一直的溪边 我默能的二 在阵的名 在你
epoch 150,perplexity 2.024866,time 0.37 sec
- 分开  说什么其 我试一够恼 我不能再想  爱有你烦 有种人直不怪我景跑活 脑在去斑简单只 学少林我环绕
- 不分开柳 你已么离 你来是带  让它什在 我暖了空蛛 相色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛
epoch 200,perplexity 1.291796,time 0.39 sec
- 分开 问候的黑不果起 虽 就底我跟棒你的那像就 你爱我以可神女 但那杂人 再来一碗热粥 配上几斤的牛肉 
- 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250,perplexity 1.176950,time 0.37 sec
- 分开 问候的人猫笑起来力哭 啦啦啦呜 用水晶球替人占卜 她说下午三点阳光射进教堂的角度 能知道你前世是狼
- 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 300,perplexity 1.143871,time 0.38 sec
- 分开  什子我看点我有你 决定中断熟悉 然后在这里 不限日期 然后将过去 慢慢温习 让我爱上你 那场悲剧
- 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 350,perplexity 1.109575,time 0.37 sec
- 分开 问弄心依身的老墙斑 在心是你在出 它每一忆